# Introduction #

For a multi-dimensional SVM, the general equation is Wx+b = *value*, where W: 1xn, x: nx1, b: 1x1. It always evaluates to a scaler result.

Note that for matrix multiplication, If A is an m×n matrix and B is an n×p matrix, the product AB is an m×p matrix.

## Perceptron ##

Basic node that is a linear boundary classifier.

![PerceptronFormulaExample.PNG](PerceptronFormulaExample.PNG)

![PerceptronNotations.PNG](PerceptronNotations.PNG)

### Perceptrons as logical operators ###

You can use perceptrons to construct the basic logical operators of AND, OR, and NOT natively. You can't do XOR though without some trickery.

Important thing to realize is that for operators like AND and OR, the weights are fighting the bias. The overall goal of a perceptron is to determine if you are on the positive or negative side of a boundary, so for something like a simple AND operator with 0's and 1's as inputs, you can have any solution like:

![AND%20example.png](AND%20example.png)

Notice the weight symmetry. You can go from an AND to an OR operator by either increasing the weights or decreasing the bias.

The NOT operator basically only has either one input or ignores all but one input. It's goal is to just return the opposite value. So something like having a weight of -1 and a bias of 0.5 works for this. Or any negative weight that's greater in magnitude than that of the positive bias.

For XOR, you need to make a multi-layer NN like the following:

![Perceptron_XOR_trick.PNG](Perceptron_XOR_trick.PNG)

### Training a Perceptron ###

If you consider the case where a hyperplane is randomly dropped to divide a space, each mischaracterized point in that space can pull the plane towards it. It does this simply by adding or subtracting itself times some weighting factor known as the learning rate (generally < 1). If the point is "above" the plane, you subtract. If it's "below", you add.

Example code for a tuning algorithm is below.

In [1]:
import numpy as np
# Setting the random seed, feel free to change it and see different solutions.
np.random.seed(42)

def stepFunction(t):
    if t >= 0:
        return 1
    return 0

def prediction(X, W, b):
    return stepFunction((np.matmul(X,W)+b)[0])

# TODO: Fill in the code below to implement the perceptron trick.
# The function should receive as inputs the data X, the labels y,
# the weights W (as an array), and the bias b,
# update the weights and bias W, b, according to the perceptron algorithm,
# and return W and b.
def perceptronStep(X, y, W, b, learn_rate = 0.01):
    # Fill in code
    for i in range(0, len(X)):
        y_pred = prediction(X[i],W,b)
        learn_rate_temp = learn_rate
        if(y[i] == y_pred):
            continue
        learn_rate_temp *= y[i]-y_pred
        W[0] += X[i][0]*learn_rate_temp
        W[1] += X[i][1]*learn_rate_temp
        b += learn_rate_temp
    return W, b
    
# This function runs the perceptron algorithm repeatedly on the dataset,
# and returns a few of the boundary lines obtained in the iterations,
# for plotting purposes.
# Feel free to play with the learning rate and the num_epochs,
# and see your results plotted below.
def trainPerceptronAlgorithm(X, y, learn_rate = 0.01, num_epochs = 25):
    x_min, x_max = min(X.T[0]), max(X.T[0])
    y_min, y_max = min(X.T[1]), max(X.T[1])
    W = np.array(np.random.rand(2,1))
    b = np.random.rand(1)[0] + x_max
    # These are the solution lines that get plotted below.
    boundary_lines = []
    for i in range(num_epochs):
        # In each epoch, we apply the perceptron step.
        W, b = perceptronStep(X, y, W, b, learn_rate)
        boundary_lines.append((-W[0]/W[1], -b/W[1]))
    return boundary_lines

# Gradient Decent #

Initial introduction and discussion is straightforward. Talked about moving in the direction that reduces error the most and warned about local minima. 

For a generalized error function, it needs to be both continuous and differentiable. On such function is logarithmic, since that would naturally assign very low error to correctly classified points and very high error to inproperly classified points, assuming that the shape was correct.

## New type of activation function ##

The first step in this process is to see what our simple step function looks like as a continuous function. A good example of that is the sigmoid function:

![SigmoidActivation.PNG](SigmoidActivation.PNG)

You can see the impact this has on the score of a linear segmenter like an SVM here:

![SigmoidOnLinear.PNG](SigmoidOnLinear.PNG)

Shown another way, we can apply this to a perceptron as the activation function, which would look like: 

![SigmoidPerceptron.PNG](SigmoidPerceptron.PNG)

## Softmax function ##

The basic sigmoid is fine and dandy for two classes. But what about many classifications? The softmax function takes care of that. It combines n exponentials to arrive at scaled probabilities for n classes. Fun fact - if n == 2, this function equates to the sigmoid function if your scores/classifications are -1 and 0.

![Softmax.PNG](Softmax.PNG)

In [2]:
import numpy as np
import math

# Write a function that takes as input a list of numbers, and returns
# the list of values given by the softmax function.
def softmax(L):
    denom = 0.
    for x in range(0, len(L)):
        denom += math.exp(L[x])
        
    valueList = []
    for x in range(0, len(L)):
        valueList.insert(len(valueList), math.exp(L[x]) / denom)
    
    return valueList

In [3]:
import numpy as np

def softmax(L):
    expL = np.exp(L)
    sumExpL = sum(expL)
    result = []
    for i in expL:
        result.append(i*1.0/sumExpL)
    return result
    
    # Note: The function np.divide can also be used here, as follows:
    # def softmax(L):
    #     expL(np.exp(L))
    #     return np.divide (expL, expL.sum())

# One-Hot Encoding - How to numerically encode unrelated things #

When making a system that tries to classify unrelated things, you don't want to have sequential enumerations between them for the learning system. Even though this works fine for humans, the machine doesn't think that way. In order to enforce separation between the items, we make row vectors that have n columns. This looks like the following:

![One-Hot.PNG](One-Hot.PNG)



# Cross Entropy and Maximum Likelihood #

Viewing things as probabilities brings up some issues. The first is that it inherently makes things a maximization problem. Generally speaking optimizers work by minimizing i.e. driving an error function to zero (such as gradient descent). The second is that calculating the conditional probability of something is a multiplicative problem that can produce very small numbers. This makes the solution ultimately very sensitive to changes in any one value, more computationally expensive, and subject to floating point issues on a computer.

The solution to this is to work in a logarithmic space. Logarithms allow us to turn a multiplication problem into an addition problem, since log(abc) = log(a) + log(b) + log(c). Also, the scale of the numbers is far more manageable, and the range is exactly what we want for an error function operating on values that have a range between 0 and 1. If we have a probability of 1 - certain and very unlikely - we have an error of 0. Likewise, if we have a probability of 0 - basically impossible and being absolutely wrong - we have an error of -infinity.

Now, we don't want to work in negative space, so we simply take the negative of the log of the probability. Also, by convention (and since we're working with exponentials), we take the sum of the negative natural logs of the probabilities.

This sum, taken across a data set with a certain classification function, is known as the cross entropy.

![crossEntropy.PNG](crossEntropy.PNG)

In [4]:
import numpy as np

# Write a function that takes as input two lists Y, P,
# and returns the float corresponding to their cross-entropy.
def cross_entropy(Y, P):
    logL = np.log(P) # exponent list if all probabilities
    yL = np.array(Y)
    crossEntropy = -np.sum(yL * logL + (1-yL)*np.log(1 - np.array(P)))
    return crossEntropy

In [5]:
import numpy as np

def cross_entropy(Y, P):
    Y = np.float_(Y)
    P = np.float_(P)
    return -np.sum(Y * np.log(P) + (1 - Y) * np.log(1 - P))

In [8]:
# Combining some formulas, we have a simplified versison here
import tensorflow as tf

softmax_data = [0.7, 0.2, 0.1]
one_hot_data = [1.0, 0.0, 0.0]

softmax = tf.placeholder(tf.float32)
one_hot = tf.placeholder(tf.float32)

# ToDo: Print cross entropy from session
cross_entropy = -tf.reduce_sum(tf.multiply(one_hot, tf.log(softmax)))

with tf.Session() as sess:
    print(sess.run(cross_entropy, feed_dict={softmax: softmax_data, one_hot: one_hot_data}))


0.35667497


For instances where we have multiple classes, the formula for cross entropy then becomes:

![multi-class_cross.PNG](multi-class_cross.PNG)

# Logistic Regression #

All of these tools combine to form the general optimization framework for learning systems known as Logistic Regression. The basic 50,000 foot outline is:

* Take your data
* Pick a random model
* Calculate the error
* Minimize the error, and obtain a better model
* Enjoy!

Sounds great, but we need a few tools. The first is the error function that we want to minimize. That's formed as follows:

![ErrorFunction.PNG](ErrorFunction.PNG)

This generalizes with the multiclass formula as follows:

![GeneralErrorFunction.PNG](GeneralErrorFunction.PNG)



# Gradient Descent #

Gradient descent, in general, involves taking the gradient of the describing error function and moving in the direction that causes the error to decrease the most. For a simple error function like a linear classifier mapped onto a sigmoid function as we've been dealing with here, it's very straight forward.

![PartialEw.gif](PartialEw.gif)

![PartialEb.gif](PartialEb.gif)

![GradE.PNG](GradE.PNG)

"The gradient is actually a scalar times the coordinates of the point! And what is the scalar? Nothing less than a multiple of the difference between the label and the prediction."

In [6]:
# Implement the following functions

# Activation (sigmoid) function
def sigmoid(x):
    return 1 / (1+np.exp(-x))

# Output (prediction) formula
def output_formula(features, weights, bias):
    return sigmoid(np.matmul(features,weights)+bias)

# Error (log-loss) formula
def error_formula(y, output):
    return -y * np.log(output) - (1 - y) * np.log(1 - output)

# Gradient descent step
def update_weights(x, y, weights, bias, learnrate):
    error = -(y - output_formula(x, weights, bias))
    weights -= learnrate * error * x
    bias -= learnrate * error
    return weights, bias

Interesting difference between the Perceptron algorithm and gradient descent is that GD listens to all points, with correctly classified ones pushing the boundaries away and incorrect ones pulling them in. Perceptron only listens to incorrectly classified points. Also, GD is a continuous method, following the "magnetic" field of the sigmoid function instead of the discrete behavior of the Perceptron algorithm.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#Some helper functions for plotting and drawing lines

def plot_points(X, y):
    admitted = X[np.argwhere(y==1)]
    rejected = X[np.argwhere(y==0)]
    plt.scatter([s[0][0] for s in rejected], [s[0][1] for s in rejected], s = 25, color = 'blue', edgecolor = 'k')
    plt.scatter([s[0][0] for s in admitted], [s[0][1] for s in admitted], s = 25, color = 'red', edgecolor = 'k')

def display(m, b, color='g--'):
    plt.xlim(-0.05,1.05)
    plt.ylim(-0.05,1.05)
    x = np.arange(-10, 10, 0.1)
    plt.plot(x, m*x+b, color)

data = pd.read_csv('../deep-learning/gradient-descent/data.csv', header=None)
X = np.array(data[[0,1]])
y = np.array(data[2])
plot_points(X,y)
plt.show()

# Implement the following functions

# Activation (sigmoid) function
def sigmoid(x):
    return 1 / (1+np.exp(-x))

# Output (prediction) formula
def output_formula(features, weights, bias):
    return sigmoid(np.matmul(features,weights)+bias)

# Error (log-loss) formula
def error_formula(y, output):
    return -y * np.log(output) - (1 - y) * np.log(1 - output)

# Gradient descent step
def update_weights(x, y, weights, bias, learnrate):
    error = -(y - output_formula(x, weights, bias))
    weights -= learnrate * error * x
    bias -= learnrate * error
    return weights, bias

np.random.seed(44)

epochs = 100
learnrate = 0.01

def train(features, targets, epochs, learnrate, graph_lines=False):
    
    errors = []
    n_records, n_features = features.shape
    last_loss = None
    weights = np.random.normal(scale=1 / n_features**.5, size=n_features)
    bias = 0
    for e in range(epochs):
        del_w = np.zeros(weights.shape)
        for x, y in zip(features, targets):
            output = output_formula(x, weights, bias)
            error = error_formula(y, output)
            weights, bias = update_weights(x, y, weights, bias, learnrate)
        
        # Printing out the log-loss error on the training set
        out = output_formula(features, weights, bias)
        loss = np.mean(error_formula(targets, out))
        errors.append(loss)
        #if e % (epochs / 10) == 0:
        #    print("\n========== Epoch", e,"==========")
        #    if last_loss and last_loss < loss:
        #        print("Train loss: ", loss, "  WARNING - Loss Increasing")
        #    else:
        #        print("Train loss: ", loss)
        #    last_loss = loss
        #    predictions = out > 0.5
        #    accuracy = np.mean(predictions == targets)
        #    print("Accuracy: ", accuracy)
        if graph_lines and e % (epochs / 100) == 0:
            display(-weights[0]/weights[1], -bias/weights[1])
            

    # Plotting the solution boundary
    plt.title("Solution boundary")
    display(-weights[0]/weights[1], -bias/weights[1], 'black')

    # Plotting the data
    plot_points(features, targets)
    plt.show()

    # Plotting the error
    plt.title("Error Plot")
    plt.xlabel('Number of epochs')
    plt.ylabel('Error')
    plt.plot(errors)
    plt.show()

train(X, y, epochs, learnrate, True)

## Important note about GD ##

Since GD as we've been looking at it here uses the sigmoid function as its activation function, it's important to note that we want all of our input values to be zero mean, with a fairly tight standard deviation (around 1 or so), since anything too far away from zero gets either squashed to zero or slammed to one.

Another thing to look out for is that as the data sets get large, you need to scale the update step i.e. learning rate by the number of data points that you have overall. This scaling of the error changes the naive SSE function to a Mean of Squared Error (MSE), which keeps the learning rate at a reasonable range (0.01 to 0.001) while also making sure that the algorithm itself stays stable and doesn't diverge in really weird ways. To see the difference between a straight-up implementation vs. one with scaling, compare the code below.

In [ ]:
import numpy as np

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1/(1+np.exp(-x))

def sigmoid_prime(x):
    """
    # Derivative of the sigmoid function
    """
    return sigmoid(x) * (1 - sigmoid(x))

learnrate = 0.5
x = np.array([1, 2, 3, 4])
y = np.array(0.5)

# Initial weights
w = np.array([0.5, -0.5, 0.3, 0.1])

### Calculate one gradient descent step for each weight
### Note: Some steps have been consilated, so there are
###       fewer variable names than in the above sample code

# TODO: Calculate the node's linear combination of inputs and weights
h = np.matmul(x,w)

# TODO: Calculate output of neural network
nn_output = sigmoid(h)

# TODO: Calculate error of neural network
error = y - nn_output

# TODO: Calculate the error term
#       Remember, this requires the output gradient, which we haven't
#       specifically added a variable for.
error_term = error * sigmoid_prime(h)

# TODO: Calculate change in weights
del_w = learnrate * error_term * x

print('Neural Network output:')
print(nn_output)
print('Amount of Error:')
print(error)
print('Change in Weights:')
print(del_w)

In [ ]:
import numpy as np
from data_prep import features, targets, features_test, targets_test


def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# TODO: We haven't provided the sigmoid_prime function like we did in
#       the previous lesson to encourage you to come up with a more
#       efficient solution. If you need a hint, check out the comments
#       in solution.py from the previous lecture.
def sigmoid_prime_x(sig):
    return sig * (1 - sig)

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # TODO: Calculate the output
        output = sigmoid(np.matmul(x,weights))

        # TODO: Calculate the error
        error = y - output

        # TODO: Calculate the error term
        error_term = error * sigmoid_prime_x(output)

        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w += error_term * x

    # TODO: Update weights using the learning rate and the average change in weights
    weights += del_w * (learnrate / len(features.values))

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

## Gradient Descent for Regression Fitting ##

Gradient descent can be readily adapted to the problem of linear regression instead of classification. The main change between the two is in the error function used. In the bi-modal classification example, the error was simply a function of how far away the point was from the correct side of the dividing line, scaled by the sigmoid function to be between 0 and 1.

For a regression, the goal is still to "float" the line to the center of the data. However the error function will seek to minimize the overall distance that all of the points are from the line. The two main error functions that we'll use for this are Sum of Absolute Difference and Sum of Square Difference (SAD and SSD). Each is scaled to the number of data points in the total set to keep the numbers in a similar range regardless of size of data set (technically, 1/m for SAD, 1/2m for SSD).

Updates are accomplished the same way as they are above, since that's the crux of gradient descent (i.e. how the error function is used to update the weights of the linear equation).

![MeanSquareError.PNG](MeanSquareError.PNG)

![AbsoluteError.PNG](AbsoluteError.PNG)

There are two ways to apply this analysis - calculate the cummulative error and then do one single update step (batch) or update the weights after each individual point evaluation (stochastic).

![BatchVsStochastic.PNG](BatchVsStochastic.PNG)

In practice, the best way to analyze large sets of points is to use the *mini-batch* method, which divides the points into small batches and treats each as a sort of stochastic step.

# Neural Networks #

Neural nets take the individual constructs from above, those simple classifiers, and combine them together again using the basic perceptron or sigmoid functions. This allows for complex regions to be formed that, while individually may be linear, are actually non-linear in shape. When you start constructing these multi-layer configurations, you end up with a number of "hidden layers" in between the inputs and the output. The general structure looks like:

![nn_hidden_layer.PNG](nn_hidden_layer.PNG)

"Each row in the matrix will correspond to the weights leading out of a single input unit, and each column will correspond to the weights leading in to a single hidden unit."

![input-times-weights.png](input-times-weights.png)



# Back Propogation #

Going forward is all well and good, but ultimately we need to be able to train the network. In order to do that, we'll need to to update the weights throughout the network. That requires a techniques known as back propagation. Regarding the notation below, error refers to the strict error - the difference between expected and calculated. Error term, however, is a shorthand notation (denoted as sigma in some of their math) for Error * f'(h), where f'(h) is the derivative of the activation function. Here the activation function is shown as being a function of the layer output (h for hidden layer output in this example). Refer to the following code segment to see how this works:

In [ ]:
import numpy as np


def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


x = np.array([0.5, 0.1, -0.2])
target = 0.6
learnrate = 0.5

weights_input_hidden = np.array([[0.5, -0.6],
                                 [0.1, -0.2],
                                 [0.1, 0.7]])

weights_hidden_output = np.array([0.1, -0.3])

## Forward pass
hidden_layer_input = np.dot(x, weights_input_hidden)
hidden_layer_output = sigmoid(hidden_layer_input)

output_layer_in = np.dot(hidden_layer_output, weights_hidden_output)
output = sigmoid(output_layer_in)

## Backwards pass
## TODO: Calculate output error
error = target - output # output error

# TODO: Calculate error term for output layer
output_error_term = error * output * (1 - output) # output error times the gradient

# TODO: Calculate error term for hidden layer
hidden_error_term = np.dot(output_error_term, weights_hidden_output) * \ # Error from the hidden layer
                    hidden_layer_output * (1 - hidden_layer_output) # Hidden layer gradient

# TODO: Calculate change in weights for hidden layer to output layer
delta_w_h_o = learnrate * output_error_term * hidden_layer_output # Here the outputs of the hidden layer are the inputs

# TODO: Calculate change in weights for input layer to hidden layer
delta_w_i_h = learnrate * hidden_error_term * x[:, None] # Same as the simple gradient decent example

print('Change in weights for hidden layer to output layer:')
print(delta_w_h_o)
print('Change in weights for input layer to hidden layer:')
print(delta_w_i_h)


Note that just like in forward propagation, we start by calculating the error between the target and the output, which is then multiplied by the gradient of the output to get the output_error_term. We then back propagate this error term by multiplying by the weight of the branch to get the hidden layer error, and multiplying that by the hidden layers gradient. Now we can update all of the weights. First, the hidden layers weight is then update by multiplying the output error term by the output of the hidden layer itself. Then ther input layer weights are update by multiplying the inputs by the error term from the hidden layer. Both are scaled by the learning rate.

The main takeaway here is that the weight of any given layer is updated by taking the error of the previous layer, multiplying it by the gradient of that layer, then multiplying that by inputs to that layer.